In [ ]:
import pandas as pd

In [ ]:
sample_submission= 'https://raw.githubusercontent.com/pinoulex/Team-IKEA-ML/main/data/sample_submission.csv'

training_data= 'https://raw.githubusercontent.com/pinoulex/Team-IKEA-ML/main/data/training_data.csv'

unlabelled_test_data= 'https://raw.githubusercontent.com/pinoulex/Team-IKEA-ML/main/data/unlabelled_test_data.csv'

fr_sentence = 'https://raw.githubusercontent.com/AlexPinel06/Team-IKEA-ML/main/data/french_sentences_realistic.csv'

In [ ]:
df_sample = pd.read_csv(sample_submission)
df_training = pd.read_csv(training_data)
df_test = pd.read_csv(unlabelled_test_data)
fr_sen = pd.read_csv(fr_sentence)

## install package

In [ ]:
!pip uninstall transformers[torch] accelerate -U
!pip install optuna transformers[torch] accelerate -U


Usage:   
  pip3 uninstall [options] <package> ...
  pip3 uninstall [options] -r <requirements file> ...

no such option: -U
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 380.1/380.1 kB 6.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.1/9.1 MB 49.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.6/302.6 kB 13.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 233.4/233.4 kB 5.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 401.2/401.2 kB 14.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.6/78.6 kB 3.1 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12

## import

In [ ]:
import pandas as pd
import numpy as np
import nltk
import random
import torch
from transformers import BertTokenizer, BertForMaskedLM, CamembertTokenizer, CamembertForSequenceClassification, AutoTokenizer, AutoModelForSequenceClassification
from nltk.corpus import wordnet
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
import re
import optuna
from transformers import Trainer, TrainingArguments
from sklearn.metrics import accuracy_score

# Dowlnoad datas for NTLK
nltk.download('wordnet')
nltk.download('omw-1.4')


[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...


True

## Initialization model & tokenizer BERT and Camembert

In [ ]:
tokenizer_bert = BertTokenizer.from_pretrained('bert-base-multilingual-uncased')
model_bert = BertForMaskedLM.from_pretrained('bert-base-multilingual-uncased')
model_bert.eval()


tokenizer_camembert = CamembertTokenizer.from_pretrained('camembert-base')


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/872k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.72M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/625 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/811k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.40M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/508 [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


## Data augmentation

In [ ]:
def synonyms(word, lang='fra'):
    synsets = wordnet.synsets(word, lang=lang)
    synonyms = set()
    for synset in synsets:
        for lemma in synset.lemma_names('fra'):
            synonyms.add(lemma.replace('_', ' '))
    return list(synonyms)

def replace_with_synonyms(sentence, lang='fra'):
    words = sentence.split()
    new_words = []
    for word in words:
        syns = synonyms(word, lang)
        if syns:
            new_word = random.choice(syns)
            new_words.append(new_word)
        else:
            new_words.append(word)
    return ' '.join(new_words)

def mask_and_predict(sentence):
    words = sentence.split()
    masked_index = random.randint(0, len(words) - 1)
    words[masked_index] = '[MASK]'
    masked_sentence = ' '.join(words)
    tokenized_text = tokenizer_bert(masked_sentence, return_tensors='pt')
    with torch.no_grad():
        outputs = model_bert(**tokenized_text)
        predictions = outputs.logits
    predicted_index = torch.argmax(predictions[0, masked_index]).item()
    predicted_token = tokenizer_bert.decode([predicted_index])
    words[masked_index] = predicted_token
    return ' '.join(words)


In [ ]:
def augment_dataframe(df):
    new_sentences = []
    for _, row in df.iterrows():
        if random.choice([True, False]):
            new_sentence = mask_and_predict(row['sentence'])
        else:
            new_sentence = replace_with_synonyms(row['sentence'])
        new_sentences.append({'id': row['id'] + 4800, 'sentence': new_sentence, 'difficulty': row['difficulty']})

    new_df = pd.DataFrame(new_sentences)
    augmented_df = pd.concat([df, new_df]).reset_index(drop=True)
    return augmented_df.sample(frac=1).reset_index(drop=True)


In [ ]:
def encode_features(df):
    label_encoder = LabelEncoder()
    df['difficulty_encoded'] = label_encoder.fit_transform(df['difficulty'])
    df['text_length'] = df['sentence'].apply(len)
    df['punctuation_count'] = df['sentence'].apply(lambda x: len(re.findall(r'[^\w\s]', x)))
    return df, label_encoder

def encode_text(data, tokenizer):
    sentences = data['sentence'].tolist()
    encodings = tokenizer(sentences, padding='max_length', truncation=True, max_length=128)
    encodings['text_length'] = torch.tensor(data['text_length'].tolist())
    encodings['punctuation_count'] = torch.tensor(data['punctuation_count'].tolist())
    return encodings


In [ ]:
df_training = augment_dataframe(df_training)
df_training, label_encoder = encode_features(df_training)
train_df, val_df = train_test_split(df_training, test_size=0.2, random_state=42)
train_encodings = encode_text(train_df, tokenizer_camembert)
val_encodings = encode_text(val_df, tokenizer_camembert)


In [ ]:
class TextDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

train_dataset = TextDataset(train_encodings, train_df['difficulty_encoded'].tolist())
val_dataset = TextDataset(val_encodings, val_df['difficulty_encoded'].tolist())


## Train model and find best parameters

In [ ]:
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return {'accuracy': accuracy_score(labels, predictions)}

def objective(trial):
    model = CamembertForSequenceClassification.from_pretrained('camembert-base', num_labels=len(label_encoder.classes_))
    training_args = TrainingArguments(
        output_dir='./results',
        num_train_epochs=6,
        per_device_train_batch_size=trial.suggest_int('per_device_train_batch_size', 4, 16),
        per_device_eval_batch_size=16,
        warmup_steps=trial.suggest_int('warmup_steps', 0, 500),
        weight_decay=trial.suggest_float('weight_decay', 0.0, 0.3),
        logging_dir='./logs',
        logging_steps=10,
        evaluation_strategy='epoch'
    )

    trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=train_dataset,
        eval_dataset=val_dataset,
        compute_metrics=compute_metrics
    )

    trainer.train()
    eval_results = trainer.evaluate()
    return eval_results['eval_accuracy']

study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=6) #change here the number of trial for optuna
best_hyperparameters = study.best_params
print(best_hyperparameters)


[I 2024-05-18 20:58:11,098] A new study created in memory with name: no-name-84dff647-7f99-4bd2-8a2b-dee786e22247


model.safetensors:   0%|          | 0.00/445M [00:00<?, ?B/s]

Some weights of CamembertForSequenceClassification were not initialized from the model checkpoint at camembert-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1474: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
<ipython-input-11-da75c3233f60>:7: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}


Epoch,Training Loss,Validation Loss,Accuracy
1,1.027800,1.098773,0.538542
2,0.747500,0.859563,0.652083
3,0.541600,0.823317,0.703646
4,0.242600,0.762854,0.764583
5,0.130700,0.891918,0.771354
6,0.055900,0.964486,0.778125


<ipython-input-11-da75c3233f60>:7: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
<ipython-input-11-da75c3233f60>:7: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
<ipython-input-11-da75c3233f60>:7: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
<ipython-input-11-da75c3233f60>:7: UserWarning: To copy construct from a 

...

[I 2024-05-18 21:27:19,652] Trial 4 finished with value: 0.7921875 and parameters: {'per_device_train_batch_size': 16, 'warmup_steps': 320, 'weight_decay': 0.2409384156563848}. Best is trial 3 with value: 0.8255208333333334.
Some weights of CamembertForSequenceClassification were not initialized from the model checkpoint at camembert-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1474: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
<ipython-input-11-da75c3233f60>:7: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_

Epoch,Training Loss,Validation Loss,Accuracy
1,1.125400,1.153215,0.536979
2,0.683100,0.845108,0.691667
3,0.462900,0.814187,0.759375
4,0.128400,1.013660,0.771875
5,0.035100,1.072059,0.803646
6,0.002500,1.057246,0.819271


<ipython-input-11-da75c3233f60>:7: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
<ipython-input-11-da75c3233f60>:7: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
<ipython-input-11-da75c3233f60>:7: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
<ipython-input-11-da75c3233f60>:7: UserWarning: To copy construct from a 

[I 2024-05-18 21:33:41,944] Trial 5 finished with value: 0.8192708333333333 and parameters: {'per_device_train_batch_size': 8, 'warmup_steps': 46, 'weight_decay': 0.06613910717664023}. Best is trial 3 with value: 0.8255208333333334.


{'per_device_train_batch_size': 6, 'warmup_steps': 65, 'weight_decay': 0.2589599175292738}


## Train model with best parameters

In [ ]:
model = CamembertForSequenceClassification.from_pretrained('camembert-base', num_labels=len(label_encoder.classes_))
training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=10,
    per_device_train_batch_size=best_hyperparameters['per_device_train_batch_size'],
    per_device_eval_batch_size=16,
    warmup_steps=best_hyperparameters['warmup_steps'],
    weight_decay=best_hyperparameters['weight_decay'],
    logging_dir='./logs',
    logging_steps=10,
    evaluation_strategy='epoch'
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    compute_metrics=compute_metrics
)

trainer.train()
results = trainer.evaluate()
print("Accuracy:", results['eval_accuracy'])


Some weights of CamembertForSequenceClassification were not initialized from the model checkpoint at camembert-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1474: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
<ipython-input-11-da75c3233f60>:7: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}


Epoch,Training Loss,Validation Loss,Accuracy
1,1.136900,1.178098,0.526563
2,0.641600,0.904258,0.653125
3,0.884100,0.876991,0.737500


<ipython-input-11-da75c3233f60>:7: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
<ipython-input-11-da75c3233f60>:7: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
<ipython-input-11-da75c3233f60>:7: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
<ipython-input-11-da75c3233f60>:7: UserWarning: To copy construct from a 

Epoch,Training Loss,Validation Loss,Accuracy
1,1.136900,1.178098,0.526563
2,0.641600,0.904258,0.653125
3,0.884100,0.876991,0.737500
4,0.481200,1.038257,0.777083
5,0.162800,1.113182,0.786458
6,0.051700,1.144596,0.809375
7,0.188600,1.078262,0.821354
8,0.104500,1.073536,0.845313
9,0.053800,1.071074,0.846354
10,0.009100,1.088358,0.854688


<ipython-input-11-da75c3233f60>:7: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
<ipython-input-11-da75c3233f60>:7: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
<ipython-input-11-da75c3233f60>:7: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
<ipython-input-11-da75c3233f60>:7: UserWarning: To copy construct from a 

Accuracy: 0.8546875


# Predict on test dataset

In [ ]:
df_t = df_test.copy()

if 'sentence' not in df_t.columns:
    raise KeyError("La colonne 'sentence' n'existe pas dans le DataFrame de test.")
if 'id' not in df_t.columns:
    df_t['id'] = np.arange(len(df_t))


df_t['text_length'] = df_t['sentence'].apply(len)
df_t['punctuation_count'] = df_t['sentence'].apply(lambda x: len(re.findall(r'[^\w\s]', x)))


test_encodings = encode_text(df_t, tokenizer_camembert)


class TestDataset(torch.utils.data.Dataset):
    def __init__(self, encodings):
        self.encodings = encodings

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        return item

    def __len__(self):
        return len(self.encodings['input_ids'])

test_dataset = TestDataset(test_encodings)


test_predictions = trainer.predict(test_dataset)


predicted_classes = np.argmax(test_predictions.predictions, axis=-1)


predicted_labels = label_encoder.inverse_transform(predicted_classes)


df_results = pd.DataFrame({
    'id': df_t['id'],
    'difficulty': predicted_labels
})


df_results.to_csv('prediction_results.csv', index=False)
print("Results saved to prediction_results.csv")


<ipython-input-24-59314630475f>:23: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}


Results saved to prediction_results.csv
